## Импорты

In [1]:
import pandas as pd
import datetime
import numpy as np

import math

## Чтение данных

In [2]:
filePath = "./ddd.csv"
df = pd.read_csv(filePath)

In [3]:
df.head()

,date
0,2020-04-02 00:00:02.425
1,2020-04-02 00:01:39.372
2,2020-04-02 00:12:28.616
3,2020-04-02 00:39:24.379
4,2020-04-02 00:41:54.129


In [4]:
df['date'] = pd.to_datetime(df.date)
df['count'] = 1

In [5]:
print(df.shape[0])
print(df.head())

1792131
                     date  count
0 2020-04-02 00:00:02.425      1
1 2020-04-02 00:01:39.372      1
2 2020-04-02 00:12:28.616      1
3 2020-04-02 00:39:24.379      1
4 2020-04-02 00:41:54.129      1


In [6]:
q = df['date']
q.iloc[-1] < datetime.datetime(2021, 9, 1)

False

In [7]:
print(df.tail())

                           date  count
1792126 2021-10-29 14:32:38.762      1
1792127 2021-10-29 14:34:11.988      1
1792128 2021-10-29 14:36:30.574      1
1792129 2021-10-29 14:38:12.585      1
1792130 2021-10-29 14:39:30.466      1


In [8]:
# убираем последние два месяца (лишние данные - после августа 2021 года)
df = df[df['date'] < datetime.datetime(2021, 9, 1)]

In [9]:
print(df.tail())

                           date  count
1716285 2021-08-31 23:51:50.038      1
1716286 2021-08-31 23:53:11.353      1
1716287 2021-08-31 23:56:11.447      1
1716288 2021-08-31 23:57:43.853      1
1716289 2021-08-31 23:58:16.200      1


# Оценка интенсивности прихода заявок в систему.

## Получаем данные за июль 2020

In [10]:
dfMy = df.copy()
dfMy = dfMy[dfMy.date >= datetime.datetime(2020, 7, 1)]
dfMy = dfMy[dfMy.date < datetime.datetime(2020, 8, 1)]

print(dfMy.size)
print(dfMy.head())

196926
                         date  count
76621 2020-07-01 00:00:14.092      1
76622 2020-07-01 00:00:33.404      1
76623 2020-07-01 00:02:17.608      1
76624 2020-07-01 00:02:18.895      1
76625 2020-07-01 00:04:33.647      1


## Выборочная дисперсия

In [11]:
def getVariance(arr, koef = 1):
    mean = np.mean(arr) * koef
    v = 0
    for x in arr:
        v += (x * koef - mean) * (x * koef - mean)
    return v / len(arr)

## 1) Постройте статистический ряд количества заявок в течение часа за три дня

In [12]:
byHours = dfMy.groupby(pd.Grouper(key="date", freq="h")).count()['count'][0:72]
byHours

date
2020-07-01 00:00:00     68
2020-07-01 01:00:00     74
2020-07-01 02:00:00    124
2020-07-01 03:00:00     15
2020-07-01 04:00:00     22
                      ... 
2020-07-03 19:00:00    149
2020-07-03 20:00:00    105
2020-07-03 21:00:00    133
2020-07-03 22:00:00    118
2020-07-03 23:00:00    218
Freq: H, Name: count, Length: 72, dtype: int64

In [13]:
mean1 = byHours.mean()
print(f'Среднее = {mean1}')
var1 = getVariance(byHours)
print(f'Дисперсия = {var1}')

Среднее = 96.76388888888889
Дисперсия = 3140.569251543211


## 2) Cтатистический ряд количества заявок в течение дня за месяц

In [14]:
byDays = dfMy.groupby(pd.Grouper(key="date", freq="d")).count()['count']
byDays

date
2020-07-01    2018
2020-07-02    2449
2020-07-03    2500
2020-07-04    3056
2020-07-05    2505
2020-07-06    2656
2020-07-07    2424
2020-07-08    2739
2020-07-09    2596
2020-07-10    2744
2020-07-11    3347
2020-07-12    2374
2020-07-13    2828
2020-07-14    2938
2020-07-15    3095
2020-07-16    2977
2020-07-17    3979
2020-07-18    3904
2020-07-19    2829
2020-07-20    3167
2020-07-21    3467
2020-07-22    3297
2020-07-23    3413
2020-07-24    4192
2020-07-25    4860
2020-07-26    3375
2020-07-27    3620
2020-07-28    4177
2020-07-29    3384
2020-07-30    3519
2020-07-31    4034
Freq: D, Name: count, dtype: int64

In [15]:
mean2 = byDays.mean() / 24
print(f'Среднее = {mean2}')
var2 = getVariance(byDays, 1 / 24)
print(f'Дисперсия = {var2}')

Среднее = 132.34274193548387
Дисперсия = 710.8009307434382


## 3.1) Воспользуйтесь данными двух месяцев, соседних Вашему варианту. Постройте статистический ряд количества заявок в течение понедельника за три месяца

In [16]:
df3Months = df.copy()
df3Months = df3Months[df3Months.date >= datetime.datetime(2020, 6, 1)]
df3Months = df3Months[df3Months.date < datetime.datetime(2020, 9, 1)]
df3Months['weekday'] = df3Months['date'].dt.dayofweek

print(df3Months.size)
print(df3Months.head())

875775
                         date  count  weekday
26087 2020-06-01 00:09:54.700      1        0
26088 2020-06-01 00:29:16.506      1        0
26089 2020-06-01 00:31:09.991      1        0
26090 2020-06-01 00:33:21.800      1        0
26091 2020-06-01 00:46:51.020      1        0


In [17]:
onlyMondays = df3Months[df3Months.weekday == 0]
byMondays = onlyMondays.groupby(pd.Grouper(key="date", freq="w")).count()['count']
byMondays

date
2020-06-07     869
2020-06-14    1260
2020-06-21    2066
2020-06-28    2082
2020-07-05    1919
2020-07-12    2656
2020-07-19    2828
2020-07-26    3167
2020-08-02    3620
2020-08-09    4039
2020-08-16    4594
2020-08-23    4259
2020-08-30    4674
2020-09-06    3509
Freq: W-SUN, Name: count, dtype: int64

In [18]:
mean3 = byMondays.mean() / 24
print(f'Среднее = {mean3}')
var3 = getVariance(byMondays, 1/24)
print(f'Дисперсия = {var3}')

Среднее = 123.63690476190476
Дисперсия = 2388.9413265306125


## 3.2) Воспользуйтесь данными двух месяцев, соседних Вашему варианту. Постройте статистический ряд количества заявок в течение воскресенья за три месяца

In [19]:
onlySundays = df3Months[df3Months.weekday == 6]
bySundays = onlySundays.groupby(pd.Grouper(key="date", freq="w")).count()['count']
bySundays

date
2020-06-07     999
2020-06-14    1460
2020-06-21    1672
2020-06-28    1643
2020-07-05    2505
2020-07-12    2374
2020-07-19    2829
2020-07-26    3375
2020-08-02    3418
2020-08-09    4272
2020-08-16    3911
2020-08-23    4198
2020-08-30    3887
Freq: W-SUN, Name: count, dtype: int64

In [20]:
mean4 = bySundays.mean() / 24
print(f'Среднее = {mean4}')
var4 = getVariance(bySundays, 1/24)
print(f'Дисперсия = {var4}')

Среднее = 117.125
Дисперсия = 2025.591613247863


## 4) Воспользуйтесь данными двух месяцев, соседних Вашему варианту. Постройте статистический ряд количества заявок в течение недели за три месяца

In [21]:
byWeeks = df3Months.groupby(pd.Grouper(key="date", freq="w")).count()['count']
byWeeks

date
2020-06-07     6445
2020-06-14    12381
2020-06-21    13210
2020-06-28    14785
2020-07-05    16241
2020-07-12    18880
2020-07-19    22550
2020-07-26    25771
2020-08-02    26748
2020-08-09    31893
2020-08-16    32189
2020-08-23    34630
2020-08-30    32693
2020-09-06     3509
Freq: W-SUN, Name: count, dtype: int64

In [22]:
mean5 = byWeeks.mean() / 7 / 24
print(f'Среднее = {mean5}')
var5 = getVariance(byWeeks, 1/7/24)
print(f'Дисперсия = {var5}')

Среднее = 124.11777210884354
Дисперсия = 3398.258554920692


## 5) Воспользуйтесь всем датасетом. Постройте статистический ряд количества заявок в течение месяца за весь период наблюдения

In [23]:
byMonths = df.groupby(pd.Grouper(key="date", freq="m")).count()['count']
byMonths

date
2020-04-30      9242
2020-05-31     16845
2020-06-30     50534
2020-07-31     98463
2020-08-31    142928
2020-09-30    116192
2020-10-31    104680
2020-11-30     78164
2020-12-31     83192
2021-01-31    100394
2021-02-28     87513
2021-03-31    109885
2021-04-30    114425
2021-05-31    141723
2021-06-30    145243
2021-07-31    152174
2021-08-31    164693
Freq: M, Name: count, dtype: int64

In [24]:
mean6 = byMonths.mean() / 30 / 24
print(f'Среднее = {mean6}')
var6 = getVariance(byMonths, 1/30/24)
print(f'Дисперсия = {var6}')

Среднее = 140.21977124183007
Дисперсия = 3588.9326011096164


## 6) Выберите среднюю интенсивность, соответствующую минимальной дисперсии

In [25]:
vars = [var1, var2, var3, var4, var5, var6]
means = [mean1, mean2, mean3, mean4, mean5, mean6]

pairs = np.array(list(zip(vars, means)))
print(pairs)

minVar, minMean = sorted(pairs, key= lambda x: x[0])[0]
print(f'Минимальная дисперсия: {minVar} со средним {minMean}')

[[3140.56925154   96.76388889]
 [ 710.80093074  132.34274194]
 [2388.94132653  123.63690476]
 [2025.59161325  117.125     ]
 [3398.25855492  124.11777211]
 [3588.93260111  140.21977124]]
Минимальная дисперсия: 710.8009307434382 со средним 132.34274193548387


# Имеется информация о начале времени записи регистрации в базу данных. Операция  записи длится в среднем 2,3 с. В системе задействовано 8 параллельных потоков.  Если  в момент прихода новой заявки все потоки заняты, то заявка становится в очередь. Таймаут ожидания обработки ответа составляет 30 с. По истечению времени таймаута необработанная заявка теряется. Аренда одного ядра (2 потока) 2400 руб. в месяц. Штраф за потерю заявки равен x руб. 
# 1.	Определите основные параметры системы. Используя статистические данные, рассчитайте среднюю интенсивность потока заявок. Найдите процент необработанных заявок. Каково среднее время, которое заявка находится в очереди? Какова средняя сумма от штрафов за месяц? Определите значение х, при котором арендовать более 20 ядер не выгодно.
# 2.	Увеличьте в три раза значение x, полученное в предыдущем пункте, и рассчитайте для него оптимальное количество используемых ядер



In [26]:
from decimal import *

n=8
print(f'Количество потоков: {n}')
t = 30 / 60 / 60
print(f't: {t}')

fst = df['date'].iloc[0]
lst = df['date'].iloc[-1]
# seconds = (lst - fst).total_seconds()
# requestInSec = seconds / df.shape[0]
# lambd = 1 / requestInSec # заявок в секунду - интенсивность
lambd = minMean
print(f'Средняя интенсивность заявок: {lambd} заявок в час')

nu = 1 / 2.3 * 60 * 60 # пропускная способность - заявок в час
print(f'Пропускная способность: {nu} заявок в час')

ro = lambd / nu
print(f'Ро: {ro}')
a = n / ro
print(f'a: {a}')
b = lambd * t
print(f'b: {b}')
c = math.exp(b * (1 - a))
print(f'c: {c}')
B = ((c - 1) / (1 - a))
print(f'B: {B}')

p0 = 0
for k in range(0, n + 1):
    p0 += ro**k / math.factorial(k)
p0 += ro**n * B / math.factorial(n)
p0**=-1
print(f'p0: {p0}')

pOtk = Decimal(ro**n) * Decimal(c) * Decimal(p0) / math.factorial(n)
print(f'pOtk: {pOtk}')
Q=1-pOtk
print(f'Q: {Q}')
lambdaEff = Decimal(lambd) * Q
print(f'lambdaEff: {lambdaEff}')
lambdaOtk = Decimal(lambd) * pOtk
print(f'lambdaOtk: {lambdaOtk}')

D = (a - c*(a+b*(a-1))) / (b*(a-1)**2)
print(f'D: {D}')

Wo = t * (ro**n / math.factorial(n)) * D * p0
print(f'Wo: {Wo}')

print('#################################################')

print(f'Процент необработанных заявок: {pOtk * Decimal(100)} %')
print(f'Среднее время заявки в очереди: {Wo} часов')
months = (lst.month + lst.year*12) - (fst.month + fst.year*12) + 1
shtrafsInMonth = pOtk * df.shape[0] / months
print(f'Среднее число штрафов в месяц: {shtrafsInMonth} штук')

Количество потоков: 8
t: 0.008333333333333333
Средняя интенсивность заявок: 132.34274193548387 заявок в час
Пропускная способность: 1565.217391304348 заявок в час
Ро: 0.08455230734767025
a: 94.61598684829305
b: 1.1028561827956989
c: 1.449703830252399e-45
B: 0.01068193621267406
p0: 0.9189235876641131
pOtk: 8.630617525819304186600988785E-59
Q: 1.000000000000000000000000000
lambdaEff: 132.3427419354838718845712719
lambdaOtk: 1.142199587963368486575831779E-56
D: 0.009789163938454915
Wo: 4.856539665049945e-18
#################################################
Процент необработанных заявок: 8.630617525819304186600988785E-57 %
Среднее время заявки в очереди: 4.856539665049945e-18 часов
Среднее число штрафов в месяц: 8.713319149052007989659653553E-54 штук


## Считаем все для 20 и 21 ядер

In [27]:
from decimal import *

n=40
print(f'Количество потоков: {n}')
t = 30 / 60 / 60
print(f't: {t}')

fst = df['date'].iloc[0]
lst = df['date'].iloc[-1]
# seconds = (lst - fst).total_seconds()
# requestInSec = seconds / df.shape[0]
# lambd = 1 / requestInSec # заявок в секунду - интенсивность
lambd = minMean
print(f'Средняя интенсивность заявок: {lambd} заявок в час')

nu = 1 / 2.3 * 60 * 60 # пропускная способность - заявок в час
print(f'Пропускная способность: {nu} заявок в час')

ro = lambd / nu
print(f'Ро: {ro}')
a = n / ro
print(f'a: {a}')
b = lambd * t
print(f'b: {b}')
c = math.exp(b * (1 - a))
print(f'c: {c}')
B = ((c - 1) / (1 - a))
print(f'B: {B}')

p0 = 0
for k in range(0, n + 1):
    p0 += ro**k / math.factorial(k)
p0 += ro**n * B / math.factorial(n)
p0**=-1
print(f'p0: {p0}')

pOtk = Decimal(ro**n) * Decimal(c) * Decimal(p0) / math.factorial(n)
print(f'pOtk: {pOtk}')
Q=1-pOtk
print(f'Q: {Q}')
lambdaEff = Decimal(lambd) * Q
print(f'lambdaEff: {lambdaEff}')
lambdaOtk = Decimal(lambd) * pOtk
print(f'lambdaOtk: {lambdaOtk}')

D = (a - c*(a+b*(a-1))) / (b*(a-1)**2)
print(f'D: {D}')

Wo = t * (ro**n / math.factorial(n)) * D * p0
print(f'Wo: {Wo}')

print('#################################################')

print(f'Процент необработанных заявок: {pOtk * Decimal(100)} %')
print(f'Среднее время заявки в очереди: {Wo} часов')
months = (lst.month + lst.year*12) - (fst.month + fst.year*12) + 1
shtrafsInMonth20 = pOtk * df.shape[0] / months
print(f'Среднее число штрафов в месяц: {shtrafsInMonth20} штук')

Количество потоков: 40
t: 0.008333333333333333
Средняя интенсивность заявок: 132.34274193548387 заявок в час
Пропускная способность: 1565.217391304348 заявок в час
Ро: 0.08455230734767025
a: 473.07993424146525
b: 1.1028561827956989
c: 7.772111343606056e-227
B: 0.002118285331501736
p0: 0.9189235876641131
pOtk: 1.064613299641380586794426671E-317
Q: 1.000000000000000000000000000
lambdaEff: 132.3427419354838718845712719
lambdaOtk: 1.408938431755231954888907929E-315
D: 0.0019247953607751857
Wo: 2.1971322093996727e-96
#################################################
Процент необработанных заявок: 1.064613299641380586794426671E-315 %
Среднее время заявки в очереди: 2.1971322093996727e-96 часов
Среднее число штрафов в месяц: 1.074814800024414757240827383E-312 штук


In [28]:
from decimal import *

n=42
print(f'Количество потоков: {n}')
t = 30 / 60 / 60
print(f't: {t}')

fst = df['date'].iloc[0]
lst = df['date'].iloc[-1]
# seconds = (lst - fst).total_seconds()
# requestInSec = seconds / df.shape[0]
# lambd = 1 / requestInSec # заявок в секунду - интенсивность
lambd = minMean
print(f'Средняя интенсивность заявок: {lambd} заявок в час')

nu = 1 / 2.3 * 60 * 60 # пропускная способность - заявок в час
print(f'Пропускная способность: {nu} заявок в час')

ro = lambd / nu
print(f'Ро: {ro}')
a = n / ro
print(f'a: {a}')
b = lambd * t
print(f'b: {b}')
c = math.exp(b * (1 - a))
print(f'c: {c}')
B = ((c - 1) / (1 - a))
print(f'B: {B}')

p0 = 0
for k in range(0, n + 1):
    p0 += ro**k / math.factorial(k)
p0 += ro**n * B / math.factorial(n)
p0**=-1
print(f'p0: {p0}')

pOtk = Decimal(ro**n) * Decimal(c) * Decimal(p0) / math.factorial(n)
print(f'pOtk: {pOtk}')
Q=1-pOtk
print(f'Q: {Q}')
lambdaEff = Decimal(lambd) * Q
print(f'lambdaEff: {lambdaEff}')
lambdaOtk = Decimal(lambd) * pOtk
print(f'lambdaOtk: {lambdaOtk}')

D = (a - c*(a+b*(a-1))) / (b*(a-1)**2)
print(f'D: {D}')

Wo = t * (ro**n / math.factorial(n)) * D * p0
print(f'Wo: {Wo}')

print('#################################################')

print(f'Процент необработанных заявок: {pOtk * Decimal(100)} %')
print(f'Среднее время заявки в очереди: {Wo} часов')
months = (lst.month + lst.year*12) - (fst.month + fst.year*12) + 1
shtrafsInMonth21 = pOtk * df.shape[0] / months
print(f'Среднее число штрафов в месяц: {shtrafsInMonth21} штук')

Количество потоков: 42
t: 0.008333333333333333
Средняя интенсивность заявок: 132.34274193548387 заявок в час
Пропускная способность: 1565.217391304348 заявок в час
Ро: 0.08455230734767025
a: 496.7339309535385
b: 1.1028561827956989
c: 3.640137147621323e-238
B: 0.002017211123871452
p0: 0.9189235876641131
pOtk: 2.070086057185148194736273014E-334
Q: 1.000000000000000000000000000
lambdaEff: 132.3427419354838718845712719
lambdaOtk: 2.739608648502973764924285964E-332
D: 0.0018327686747567147
Wo: 8.685544724001194e-102
#################################################
Процент необработанных заявок: 2.070086057185148194736273014E-332 %
Среднее время заявки в очереди: 8.685544724001194e-102 часов
Среднее число штрафов в месяц: 2.089922352403704703025834124E-329 штук


In [29]:
x = 2400 / (shtrafsInMonth20 - shtrafsInMonth21)
print(f'x: {x}')
x3 = x * 3
print(f'x3: {x3}')

x: 2.232942828797559625685311085E+315
x3: 6.698828486392678877055933255E+315


In [30]:
mins = []
for n2 in range(40, -1, -1):
    n = n2 * 2
    t = 30 / 60 / 60

    fst = df['date'].iloc[0]
    lst = df['date'].iloc[-1]
    # seconds = (lst - fst).total_seconds()
    # requestInSec = seconds / df.shape[0]
    # lambd = 1 / requestInSec # заявок в секунду - интенсивность
    lambd = minMean

    nu = 1 / 2.3 * 60 * 60 # пропускная способность - заявок в час

    ro = lambd / nu
    a = n / ro
    b = lambd * t
    c = math.exp(b * (1 - a))
    B = ((c - 1) / (1 - a))

    p0 = 0
    for k in range(0, n + 1):
        p0 += ro**k / math.factorial(k)
    p0 += ro**n * B / math.factorial(n)
    p0**=-1

    pOtk = Decimal(ro**n) * Decimal(c) * Decimal(p0) / math.factorial(n)
    Q=1-pOtk
    lambdaEff = Decimal(lambd) * Q
    lambdaOtk = Decimal(lambd) * pOtk

    D = (a - c*(a+b*(a-1))) / (b*(a-1)**2)

    Wo = t * (ro**n / math.factorial(n)) * D * p0

    months = (lst.month + lst.year*12) - (fst.month + fst.year*12) + 1
    shtrafsInMonth = pOtk * df.shape[0] / months
    price = 2400 * n2
    print(f'{n2} ядер: штрафов = {shtrafsInMonth}, сумма штрафа = {shtrafsInMonth*x3}, цена ядер = {price}, штрафы + ядра = {shtrafsInMonth*x3 + price}')
    mins.append(shtrafsInMonth*x3 + price)

pairs = np.array(list(zip(mins, range(40, -1, -1))))

minCost, minN = sorted(pairs, key= lambda x: x[0])[0]
print(f'{minCost} при {minN} ядрах')

40 ядер: штрафов = 0E-390, сумма штрафа = 0E-102, цена ядер = 96000, штрафы + ядра = 96000.00000000000000000000000
39 ядер: штрафов = 0E-381, сумма штрафа = 0E-93, цена ядер = 93600, штрафы + ядра = 93600.00000000000000000000000
38 ядер: штрафов = 0E-375, сумма штрафа = 0E-87, цена ядер = 91200, штрафы + ядра = 91200.00000000000000000000000
37 ядер: штрафов = 0E-368, сумма штрафа = 0E-80, цена ядер = 88800, штрафы + ядра = 88800.00000000000000000000000
36 ядер: штрафов = 0E-355, сумма штрафа = 0E-67, цена ядер = 86400, штрафы + ядра = 86400.00000000000000000000000
35 ядер: штрафов = 0E-354, сумма штрафа = 0E-66, цена ядер = 84000, штрафы + ядра = 84000.00000000000000000000000
34 ядер: штрафов = 0E-347, сумма штрафа = 0E-59, цена ядер = 81600, штрафы + ядра = 81600.00000000000000000000000
33 ядер: штрафов = 0E-338, сумма штрафа = 0E-50, цена ядер = 79200, штрафы + ядра = 79200.00000000000000000000000
32 ядер: штрафов = 0E-331, сумма штрафа = 0E-43, цена ядер = 76800, штрафы + ядра = 768